In [15]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from graphviz import Source,render

In [16]:
def construct(data,question,parent,user):
    if parent == ['root']:
        root =  data['quests'][0]["dataquestionnaire"]
    else : 
        root = data
    if user == '':
        try : 
            user = data['logs'][0]['IDuser']

        except :
            if "unknow" not in question :
                user = "unknow"
            else :
                user = 'unknow'
                i = 0
                while user in question :
                    i+=1
                    user = 'unknow{}'.format(i) 
        question[user] = {}


    for element in root:
        typeQ = element['answerType']

        if typeQ not in question[user] :
            question[user][typeQ] = {}

        question[user][typeQ][element['code']] = {}

        if typeQ == "Range":
            if 'sousquest' in element['options'] :
                question[user][typeQ][element['code']] = {
                    'value' : element['value'],
                    'subtitle':element['subtitle'],
                    'subquestions':{},
                    'answered':element['answered'],
                    'root' : parent}  

                parent.append(element['code'])
                for subquestion in element['options']['sousquest'] :
                    construct([subquestion],question,parent,user)

            else :
                question[user][typeQ][element['code']] = {
                    'answered':element['answered'],
                    'value' : element['value'],
                    'min-max' : [element['options']['min'] ,element['options']['max']],
                    'root':parent}
            parent = ['root']
            


        if typeQ == "Radio" :
            parent.append(element['code'])
            for subquestion in element['questionOptions'] :
                question[user][typeQ][element['code']][subquestion['text']] = { 
                    'answered':element['answered'],
                    "checked": subquestion['checked'] ,
                    "value": subquestion['value'],
                    "root": parent}
            parent = ['root']


        if typeQ == 'Checkbox' :
            parent.append(element['code'])
            for subquestion in element['questionOptions'] :
                question[user][typeQ][element['code']][subquestion['text']] = { 
                    'answered':element['answered'],
                    "checked": subquestion['checked'] ,
                    "value": subquestion['value'],
                    "root": parent}
            parent = ['root']
      
        if typeQ == 'Text' :
            question[user][typeQ][element['code']] = {
                'answered':element['answered'],
                'value' : element["value"],
                'root' : parent
                }
            parent = ['root']

        if typeQ == 'YesNo':
            if 'sousquest' in element['options'] :
                question[user][typeQ][element['code']] = {
                    'answered':element['answered'],
                    'value' : element['checked'],
                    'subquestion_name' : {element['options']['sousquest'][0]['code']},
                    'subquestions':{},
                    'subtitle' :element['options']['sousquest'][0]['subtitle'],
                    'root' : parent }
                
                parent.append(element['code'])
                for subquestion in element['options']['sousquest'] :
                    construct([subquestion],question,parent,user)

            else :
                question[user][typeQ][element['code']] = {
                    'answered':element['answered'],
                    'value' : element['checked'],
                    'root' : parent }
            parent = ['root']


        if typeQ == 'Contact':
            question[user][typeQ][element['code']] = {
                'answered':element['answered'],
                'subtitle' : element['subtitle'],
                'value' : element['value'],
                'root' : parent }
            parent = ['root']

                
        if typeQ == 'Time':
            question[user][typeQ][element['code']] = {
                'subtitle' : element['subtitle'],
                'value' : element['value'],
                'root' : parent }
            parent = ['root']

        if typeQ == 'Input':
            question[user][typeQ][element['code']] = {
                'answered':element['answered'],
                'subtitle' : element['subtitle'],
                'value' : element['value'],
                'root' : parent }
            parent = ['root']


        if typeQ == 'Section':
            if 'sousquest' in element['options'] :
                question[user][typeQ][element['code']] = {
                    'answered':element['answered'],
                    'subtitle' :element['subtitle'],
                    'root' : parent }

                parent.append(element['code'])
                for subquestion in element['options']['sousquest'] :
                    construct([subquestion],question,parent,user)
            parent = ['root']

    return question


In [17]:
question = {}
files = 'bcc08.json'
with open("json/{}".format(files), 'r',encoding='utf8') as f:
    datastore = json.load(f)
    if "quests" in datastore :
        if datastore["quests"] != []:
            question = construct(datastore,question,['root'],'')

In [18]:
def my_function(x):
  return  list(set(tuple(element) for element in x))

In [19]:
def retrieve_path(tree):
    path = []
    for personne in tree.keys() :
        #Personne
        for type_q in tree[personne].keys() :
            #Type de question
            if type_q not in ["Checkbox","Radio"] :
                #si question est checkbox ou radio
                for quest in tree[personne][type_q].keys() :
                    # Question
                    #print(question)
                    root = tree[personne][type_q][quest]['root']
                    path.append(root)

            else : 
                for quest in tree[personne][type_q].keys() :
                    for subelement in tree[personne][type_q][quest].keys():

                        root = tree[personne][type_q][quest][subelement]['root']
                        path.append(root)
    print(path)
    return my_function(path)

root = retrieve_path(question)

[['root', 'matin_bien_dormi', 'matin_mal_dormi_cause'], ['root', 'q5_emma_douleur_morale'], ['root', 'q5_emma_douleur_morale'], ['root', 'q5_emma_douleur_morale'], ['root', 'q5_emma_douleur_morale'], ['root'], ['root'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root', 'q18_evenement_impact', 'q19_evenement_impact_envie'], ['root'], ['root', 'soir_douleurs_physiques'], ['root', 'soir_douleurs_physiques'], ['root'], ['root'], ['root'], ['root'], ['root'], ['root', 'matin_bien_dormi', 'matin_mal_dormi_cause'], ['root', 'matin_bien_dormi', 'matin_mal_dormi_cause'], ['root', 'matin_bien_dormi', 'matin_mal_dormi_cause'], ['root', 'matin_bien_dormi', 'matin_mal_dormi_cause'], ['root', 'matin_bien_dormi', 'matin_mal_do

In [20]:
J = []
for path in root :
    parts = path
    parent_list, current_list = J, J

    for index, part in enumerate(parts):
        for item in current_list:
            if part in item and type(item) != str:
                parent_list, current_list = current_list, item[part]
                break
        else:
            if index == len(parts) - 1:
                current_list.append(part)
            else:
                new_list = []
                current_list.append({part:new_list})
                parent_list, current_list = current_list, new_list 

In [21]:
udo = Node("root")
def recur(arbre,previous_root):
    for branche in arbre :
        if type(branche) == str :
            Node(branche,parent = previous_root)
        elif type(branche) == dict :
            for key in branche.keys() :
                tmp = Node(key,parent = previous_root)
                recur(branche[key],tmp) 


recur(J[0]['root'],udo)

In [22]:
name = '{}.dot'.format(files)
DotExporter(udo).to_dotfile(name)
Source.from_file(name)
render('dot', 'png', name) 

'bcc08.json.dot.png'